In [2]:
import torch
import commons
import utils
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence
from scipy.io.wavfile import write
import IPython.display as ipd

def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

CONFIG_PATH = "./configs/vits2_ljs_ring.json"
hps = utils.get_hparams_from_file(CONFIG_PATH)

MODEL_PATH = "./logs/test/G_350000.pth"
    
if (
    "use_mel_posterior_encoder" in hps.model.keys()
    and hps.model.use_mel_posterior_encoder == True
):
    print("Using mel posterior encoder for VITS2")
    posterior_channels = 80  # vits2
    hps.data.use_mel_posterior_encoder = True
else:
    print("Using lin posterior encoder for VITS1")
    posterior_channels = hps.data.filter_length // 2 + 1
    hps.data.use_mel_posterior_encoder = False

net_g = SynthesizerTrn(
    len(symbols),
    posterior_channels,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model
).cuda()
_ = net_g.eval()
_ = utils.load_checkpoint(MODEL_PATH, net_g, None)

Using mel posterior encoder for VITS2
INFO:root:Loaded checkpoint './logs/test/G_350000.pth' (iteration 1759)


In [5]:
# "dec" 모듈의 파라미터 개수 계산
dec_module = getattr(net_g, 'dec', None)
if dec_module is not None:
    total_params = sum(param.numel() for param in dec_module.parameters())
    print(f"Total number of parameters in 'dec': {total_params}")
else:
    print("Module 'dec' not found in net_g.")

Total number of parameters in 'dec': 30104452


In [7]:
# "dec" 모듈 내부의 모든 서브모듈 출력
if dec_module is not None:
    print("All submodules in 'dec':")
    for name, layer in dec_module.named_modules():  # 모든 하위 모듈 (재귀적)
        print(f"Name: {name}, Type: {type(layer)}")
else:
    print("Module 'dec' not found in net_g.")

All submodules in 'dec':
Name: , Type: <class 'models.Generator'>
Name: conv_pre, Type: <class 'torch.nn.modules.conv.Conv1d'>
Name: ups, Type: <class 'torch.nn.modules.container.ModuleList'>
Name: ups.0, Type: <class 'torch.nn.modules.conv.ConvTranspose1d'>
Name: ups.1, Type: <class 'torch.nn.modules.conv.ConvTranspose1d'>
Name: alphas, Type: <class 'torch.nn.modules.container.ParameterList'>
Name: resblocks, Type: <class 'torch.nn.modules.container.ModuleList'>
Name: resblocks.0, Type: <class 'modules.ResBlock1'>
Name: resblocks.0.convs1, Type: <class 'torch.nn.modules.container.ModuleList'>
Name: resblocks.0.convs1.0, Type: <class 'torch.nn.modules.conv.Conv1d'>
Name: resblocks.0.convs1.1, Type: <class 'torch.nn.modules.conv.Conv1d'>
Name: resblocks.0.convs1.2, Type: <class 'torch.nn.modules.conv.Conv1d'>
Name: resblocks.0.convs2, Type: <class 'torch.nn.modules.container.ModuleList'>
Name: resblocks.0.convs2.0, Type: <class 'torch.nn.modules.conv.Conv1d'>
Name: resblocks.0.convs2.1,

In [9]:
import time
import numpy as np

num_iterations = 1
inference_times = []
rtf_values = []
TEXTS = ["The future belongs to those who believe in the beauty of their dreams."]

for idx, text in enumerate(TEXTS):
    stn_tst = get_text(text, hps)
    with torch.no_grad():
        x_tst = stn_tst.cuda().unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()

        for _ in range(num_iterations):
            start = time.perf_counter()
            audio = (
                net_g.infer(
                    x_tst, x_tst_lengths, noise_scale=0.667, noise_scale_w=0.8, length_scale=1
                )[0][0, 0]
                .data.cpu()
                .float()
                .numpy()
            )
            
            end = time.perf_counter()
            inference_time = end - start
            inference_times.append(inference_time)
            
            audio_length = len(audio) / hps.data.sampling_rate
            rtf = inference_time / audio_length
            rtf_values.append(rtf)
            
            ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=True))
            # write(data=audio, rate=hps.data.sampling_rate, filename=f'results.wav')